In [26]:
import uproot
import numpy as np
import awkward as ak

import vector

import hist

def spacial_invert(vec):
    return -vec.to_Vector3D().to_Vector4D()+vector.obj(px=0, py=0, pz=0, E=vec.E)



In [27]:
filename = "./input/MC/Signal/019426EE-3D50-1249-B266-F6DBA0AFE3B5.root"

file = uproot.open(filename)

In [28]:
file.classnames()

{'tag;1': 'TObjString',
 'Events;1': 'TTree',
 'LuminosityBlocks;1': 'TTree',
 'Runs;1': 'TTree',
 'MetaData;1': 'TTree',
 'ParameterSets;1': 'TTree'}

In [29]:
tree = file['Events']

In [30]:
filter_names = ['/(Electron|Muon|Jet)_(pt|eta|phi|mass)/',
                '/n(Muon|Electron|Jet)/',
                '/(Electron|Muon)_charge/',
                '/Muon_(triggerIdLoose)/',
                '/Muon_(pfRelIso04_all|tightId)/',
                '/HLT_(IsoMu24|Ele32_WPTight_Gsf)/',
                'Electron_mvaFall17V2Iso_WP90',
                '/Jet_(btagDeepB|btagDeepFlavB)/',
                'genWeight']
mc_filter_names = ['/(Electron|Muon)_genPart(Idx|Flav)/',
                '/GenPart_(pdgId|genPartIdxMother)/',
                'nGenPart']

In [32]:

outfile = uproot.recreate('outfile.root')
outfile.mktree('tout', {
    "N_jet_loose": "var*int64",
    "N_jet_medium": "var*int64",
    "N_jet_tight": "var*int64",
})
## define histograms

h_Muon_pt = hist.Hist(hist.axis.Regular(bins=100,start=0,stop=200,name="h_Muon_pt"))
h_Muon_eta = hist.Hist(hist.axis.Regular(bins=100,start=-5,stop=5,name="h_Muon_eta"))
h_Electron_pt = hist.Hist(hist.axis.Regular(bins=100,start=0,stop=200, name="h_Electron_pt",)) 
h_Electron_eta = hist.Hist(hist.axis.Regular(bins=100,start=-5,stop=5, name="h_Electron_eta",))

h_Muon_pt_weighted = hist.Hist(hist.axis.Regular(bins=100,start=0,stop=200,name="h_Muon_pt_weighted"))
h_Muon_eta_weighted = hist.Hist(hist.axis.Regular(bins=100,start=-5,stop=5,name="h_Muon_eta_weighted"))
h_Electron_pt_weighted = hist.Hist(hist.axis.Regular(bins=100,start=0,stop=200,name="h_Electron_pt_weighted"))
h_Electron_eta_weighted = hist.Hist(hist.axis.Regular(bins=100,start=-5,stop=5,name="h_Electron_eta_weighted"))

h_Muon_Electron_invariant_mass = hist.Hist(hist.axis.Regular(bins=100,start=12,stop= 412,name="Muon_Electron_invariant_mass"))
h_Muon_Electron_invariant_mass_weighted = hist.Hist(hist.axis.Regular(bins=100,start=12,stop= 412,name="Muon_Electron_invariant_mass_weighted"))

h_leading_lepton_pt = hist.Hist(hist.axis.Regular(bins=45,start=20,stop=200,name="leading_lepton_pt"))
h_leading_lepton_pt_weighted = hist.Hist(hist.axis.Regular(bins=45,start=20,stop=200,name="leading_lepton_pt_weighted"))


In [8]:
def skimming(mc_flag, lumi=None, xs=None):
    # ^ is xor in python
    trigger_cut = "HLT_IsoMu24 | HLT_Ele32_WPTight_Gsf"
    btag_deepflav_wp = {'loose': 0.0490, 'medium': 0.2783, 'tight': 0.71}
    # get the sumW from the runs tree
    if mc_flag:
        n_gen = file['Runs']['genEventCount'].array()
        Sum_W = file['Runs']['genEventSumw'].array()
    for events in tree.iterate(
        filter_name=filter_names+mc_filter_names,cut = trigger_cut,
        entry_stop=100000):
        ## apply muon and electron cuts 
        m_pt_cut = events['Muon_pt']>27
        m_eta_cut = np.abs(events['Muon_eta'])<2.4
        m_iso_cut = events['Muon_pfRelIso04_all']<0.15
        m_tight_cut = events["Muon_tightId"]

        muon_cuts = (m_pt_cut) &(m_eta_cut) & (m_iso_cut)&m_tight_cut
        muon_idxs = ak.local_index(muon_cuts)[muon_cuts]
        muon_idxs = muon_idxs[ak.num(muon_idxs)>0]
        events = events[ak.any(muon_cuts, axis=1)]
        events['Muon_Idx'] = muon_idxs

        e_pt_cut =  events['Electron_pt']>35
        e_eta_cut = np.abs(events['Electron_eta'])<2.4
        e_iso_cut =  events["Electron_mvaFall17V2Iso_WP90"]

        electron_cuts = (e_pt_cut)&(e_eta_cut)&(e_iso_cut)
        electron_idxs = ak.local_index(electron_cuts)[electron_cuts]
        electron_idxs = electron_idxs[ak.num(electron_idxs)>0]
        events = events[ak.any(electron_cuts, axis=1)]
        events["Electron_Idx"] = electron_idxs

        b_cut = events['Jet_btagDeepFlavB']>btag_deepflav_wp['medium']
        events = events[ak.any(b_cut, axis=1)]
        # make sure that muon and electron are of opposite charge
        # this cut is applied after the others because you can only 
        # multiply two arrays if they both have at least one entry along the first dim
        # the pt cuts make sure of this: we only have events left where we have 
        # at least one electron and at leats one muon
        charge_cut = (events['Muon_charge'][:, 0] * events['Electron_charge'][:, 0]) < 0
        events = events[charge_cut]
        # create four-vectors
        # objects are ordered by pt so we take index 0 for highest pt
        mu_pt = events['Muon_pt', events['Muon_Idx']]
        mu_eta = events['Muon_eta', events['Muon_Idx']]
        mu_phi = events['Muon_phi', events['Muon_Idx']]
        mu_mass = events['Muon_mass', events['Muon_Idx']]
        electron_pt = events['Electron_pt', events['Electron_Idx']]
        electron_eta = events['Electron_eta', events['Electron_Idx']]
        electron_phi = events['Electron_phi', events['Electron_Idx']]
        electron_mass = events['Electron_mass', events['Electron_Idx']]
        muon_4d = vector.array({'pt':mu_pt[:, 0],
                                'eta': mu_eta[:, 0], 
                                'phi':mu_phi[:, 0],
                                'mass': mu_mass[:, 0]})
        electron_4d = vector.array({'pt':electron_pt[:, 0],
                                    'eta': electron_eta[:, 0],
                                    'phi':electron_phi[:, 0],
                                    'mass': electron_mass[:, 0]})
        # calculate the deltaR
        # cut out events with deltaR < 0.4
        delta_r_cut = muon_4d.deltaR(electron_4d) > 0.4
        muon_4d = muon_4d[delta_r_cut]; electron_4d=electron_4d[delta_r_cut]
        events = events[delta_r_cut]
        # how many jets would we have if we took other wp's?
        N_jet_loose = ak.sum(events['Jet_btagDeepFlavB']>btag_deepflav_wp['loose'], axis=1)
        N_jet_medium = ak.sum(events['Jet_btagDeepFlavB']>btag_deepflav_wp['medium'], axis=1)
        N_jet_tight = ak.sum(events['Jet_btagDeepFlavB']>btag_deepflav_wp['tight'], axis=1)
        # first drop events where there is no jet that passes the wp
        b_tag_medium_cut = ak.any(events['Jet_btagDeepFlavB']>btag_deepflav_wp['medium'], axis=1)
        events = events[b_tag_medium_cut]
        # now our event has at least one jet passing the wp
        # event could look like this: [not_passing, not_passing, passing, not_passing, passing]
        # we want the first one that passes (index = 2)


        ## unused right now ##

        #b_tag_medium_cut = events['Jet_btagDeepFlavB']>btag_deepflav_wp['medium']
        #jet_4d = vector.array({'pt':events['Jet_pt', b_tag_medium_cut][:, 0],
                                #'eta': events['Jet_eta', b_tag_medium_cut][:, 0], 
                                #'phi':events['Jet_phi', b_tag_medium_cut][:, 0],
                                #'mass': events['Jet_mass', b_tag_medium_cut][:, 0]})
        #opposite_jet = spacial_invert(jet_4d)
        #dR_mu_jet = muon_4d.deltaR(jet_4d)
        #dR_e_jet = electron_4d.deltaR(jet_4d)
        #dR_mu_e = muon_4d.deltaR(electron_4d)

        if mc_flag:
            weight = events['genWeight']*lumi*xs


        # fill the histograms 





In [51]:

correction_file = "./corrections/muon_Z.json.gz"
if correction_file.endswith(".json.gz"):
    import gzip
    with gzip.open(correction_file, 'rt') as file:
        data = file.read().strip()
        ceval = correctionlib.CorrectionSet.from_string(data)
else:
    ceval = correctionlib.CorrectonSet.from_file(correction_file)


list(ceval.keys())

['NUM_GlobalMuons_DEN_genTracks',
 'NUM_HighPtID_DEN_TrackerMuons',
 'NUM_HighPtID_DEN_genTracks',
 'NUM_IsoMu24_DEN_CutBasedIdTight_and_PFIsoTight',
 'NUM_LooseID_DEN_TrackerMuons',
 'NUM_LooseID_DEN_genTracks',
 'NUM_LooseRelIso_DEN_LooseID',
 'NUM_LooseRelIso_DEN_MediumID',
 'NUM_LooseRelIso_DEN_MediumPromptID',
 'NUM_LooseRelIso_DEN_TightIDandIPCut',
 'NUM_LooseRelTkIso_DEN_HighPtIDandIPCut',
 'NUM_LooseRelTkIso_DEN_TrkHighPtIDandIPCut',
 'NUM_MediumID_DEN_TrackerMuons',
 'NUM_MediumID_DEN_genTracks',
 'NUM_MediumPromptID_DEN_TrackerMuons',
 'NUM_MediumPromptID_DEN_genTracks',
 'NUM_Mu50_or_OldMu100_or_TkMu100_DEN_CutBasedIdGlobalHighPt_and_TkIsoLoose',
 'NUM_SoftID_DEN_TrackerMuons',
 'NUM_SoftID_DEN_genTracks',
 'NUM_TightID_DEN_TrackerMuons',
 'NUM_TightID_DEN_genTracks',
 'NUM_TightRelIso_DEN_MediumID',
 'NUM_TightRelIso_DEN_MediumPromptID',
 'NUM_TightRelIso_DEN_TightIDandIPCut',
 'NUM_TightRelTkIso_DEN_HighPtIDandIPCut',
 'NUM_TightRelTkIso_DEN_TrkHighPtIDandIPCut',
 'NUM_Tra

In [52]:
for ix in ceval['NUM_IsoMu24_DEN_CutBasedIdTight_and_PFIsoTight'].inputs:
    print(f"   Input {ix.name} ({ix.type}): {ix.description}")

   Input year (string): year/scenario: example 2016preVFP, 2017 etc
   Input abseta (real): Probe abseta
   Input pt (real): Probe pt
   Input ValType (string): sf or syst (currently 'sf' is nominal, and 'systup' and 'systdown' are up/down variations with total stat+syst uncertainties. Individual systs are also available (in these cases syst only, not sf +/- syst)


In [54]:
events['Muon_pt'][:, 0]

<Array [77.7, 29, 111, ... 64.6, 38.5, 35.9] type='6071 * float32'>

In [61]:
ceval['NUM_IsoMu24_DEN_CutBasedIdTight_and_PFIsoTight'].evaluate('2018_UL',np.abs(events['Muon_eta'][:, 0]), events['Muon_pt'][:, 0], 'sf')

array([0.97903552, 0.97308885, 0.97903552, ..., 0.97903552, 1.00731044,
       1.00731044])

In [42]:
import correctionlib
correction_file = "./corrections/puWeights.json.gz"
if correction_file.endswith(".json.gz"):
    import gzip
    with gzip.open(correction_file, 'rt') as file:
        data = file.read().strip()
        ceval = correctionlib.CorrectionSet.from_string(data)
else:
    ceval = correctionlib.CorrectonSet.from_file(correction_file)

In [43]:
list(ceval.keys())

['Collisions18_UltraLegacy_goldenJSON']

In [44]:
for corr in ceval.values():
    print(f"Correction {corr.name} has {len(corr.inputs)} inputs")
    for ix in corr.inputs:
        print(f"   Input {ix.name} ({ix.type}): {ix.description}")

Correction Collisions18_UltraLegacy_goldenJSON has 2 inputs
   Input NumTrueInteractions (real): Number of true interactions
   Input weights (string): nominal, up, or down


In [49]:
tree['Pileup_nTrueInt'].array()

<Array [20, 25, 30, 33, 4, ... 34, 44, 40, 41] type='1386000 * float32'>

In [50]:
ceval['Collisions18_UltraLegacy_goldenJSON'].evaluate(tree['Pileup_nTrueInt'].array(), "nominal")

array([0.94641737, 0.98739476, 0.99822759, ..., 1.08183655, 1.03718   ,
       1.04711103])

In [9]:
file.classnames()

{'tag;1': 'TObjString',
 'Events;1': 'TTree',
 'LuminosityBlocks;1': 'TTree',
 'Runs;1': 'TTree',
 'MetaData;1': 'TTree',
 'ParameterSets;1': 'TTree'}

In [20]:
file['Runs'].show()

name                 | typename                 | interpretation                
---------------------+--------------------------+-------------------------------
run                  | uint32_t                 | AsDtype('>u4')
genEventCount        | int64_t                  | AsDtype('>i8')
genEventSumw         | double                   | AsDtype('>f8')
genEventSumw2        | double                   | AsDtype('>f8')
nLHEScaleSumw        | uint32_t                 | AsDtype('>u4')
LHEScaleSumw         | double[]                 | AsJagged(AsDtype('>f8'))
nLHEPdfSumw          | uint32_t                 | AsDtype('>u4')
LHEPdfSumw           | double[]                 | AsJagged(AsDtype('>f8'))


In [25]:
file['Runs']['genEventCount'].array()

<Array [1386000] type='1 * int64'>

In [23]:
file['Runs']['genEventSumw'].array(), file['Runs']['genEventSumw2'].array()

(<Array [9.99e+07] type='1 * float64'>, <Array [7.33e+09] type='1 * float64'>)

In [ ]:
def is_from_pdg(pdg_id, gen_idx, pdg_ids, mother_idxs):
    """
    Function to check if a generated daughter 
    particle originates from a mother particle 

    Parameters
    ----------
    pdg_id: (int) PDGId of the mother particle,
    gen_idx: (int) idx of the daughter particle,
    pdg_ids: array of pdg_ids,
    mother_idxs: array containing indices of where the
                 mother particle is located.

    Returns
    -------
    bool: is_from_pdg
    """
    # if a muon is misidentified it may have an idx of -1
    if gen_idx <= 0:
        return False
    mother_idx = mother_idxs[gen_idx]
    if pdg_ids[mother_idx] == pdg_id:
        return True
    elif pdg_ids[mother_idx] == pdg_ids[gen_idx]:
        return is_from_pdg(pdg_id, mother_idx, pdg_ids, mother_idxs)
    else:
        return False

In [47]:
tree.show("*PU*")

name                 | typename                 | interpretation                
---------------------+--------------------------+-------------------------------
Pileup_nPU           | int32_t                  | AsDtype('>i4')


In [89]:
events['Muon_tightId']

<Array [[True], [], ... [True, False]] type='540 * var * bool'>

In [ ]:
events['Jet_']